<a href="https://colab.research.google.com/github/DavidVollendroff/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
# Engineer at least two new features.
df['bed_bath_ratio']  = (df['bedrooms'] / df['bathrooms'])
df['pet_friendly'] = (df['cats_allowed'] == 1) & (df['dogs_allowed'] == 1)

In [0]:
# Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

# string to datetime conversion
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=False)

# function to extract month
def month(date_time):
  return date_time.month

df['created'] = df['created'].apply(month)

df = df[(df['bathrooms'] != 0)] # removing 0 bathroom condos

df.pet_friendly = df.pet_friendly.astype(int) # casting booleans to ints

train = df[(df['created'] < 6)] # split by month for training

test = df[(df['created'] == 6)] # split by month for testing

In [5]:
# Fit a linear regression model with at least two features.

from sklearn.linear_model import LinearRegression

model = LinearRegression()

features = ['bathrooms', 'bedrooms', 'bed_bath_ratio', 'pet_friendly']
labels = 'price'

model.fit(train[features], train[labels])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# Get the model's coefficients and intercept.
coef = model.coef_
intercept = model.intercept_

In [0]:
# Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

train_pred = model.predict(train[features])
test_pred = model.predict(test[features])
train_labels = train[labels]
test_labels = test[labels]

train_mae = mean_absolute_error(train_labels, train_pred)
train_mse = mean_squared_error(train_labels, train_pred)
train_r2 = r2_score(train_labels, train_pred)

test_mae = mean_absolute_error(test_labels, test_pred)
test_mse = mean_squared_error(test_labels, test_pred)
test_r2 = r2_score(test_labels, test_pred)


In [12]:
string = '{} errors:\nmae = {}, mse = {}, r_score = {}'
print(string.format('Training', train_mae, train_mse, train_r2))
print(string.format('Testing', test_mae, test_mse, test_r2))

Training errors:
mae = 796.5252722612234, mse = 1441703.9005300913, r_score = 0.5346644891073173
Testing errors:
mae = 806.456452209694, mse = 1416010.5573027518, r_score = 0.5445265842637952
